we do it today

In [ ]:
print(final_model.summary(expand_nested=True, show_trainable=True))

NameError: name 'final_model' is not defined

In [ ]:
# Cell 1: Dynamic Class Detection, Setup, and YOLO Data Preparation

import os
import shutil
from google.colab import drive
from sklearn.model_selection import train_test_split
import glob
import random
import yaml
import numpy as np
import tensorflow as tf
from pathlib import Path

# --- 1. Mount Google Drive ---
drive.mount('/content/drive')

# --- 2. Configuration & Paths ---
# Source directory for your dataset
DATA_DIR = Path('/content/drive/MyDrive/final data ')
# Target directory for YOLO-style structure (local, temporary)
YOLO_DATA_ROOT = '/content/yolo_dataset'
# Folder path to save the final Keras model
MODEL_SAVE_PATH = '/content/drive/MyDrive/mod/animal_classifier_9class.keras'

# Set Random Seeds for Reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
VALIDATION_SPLIT = 0.2

# --- 3. DYNAMICALLY Determine Class Names ---
# This finds all subdirectories in DATA_DIR and uses them as class names
NEW_CLASS_NAMES = sorted([d.name for d in DATA_DIR.iterdir() if d.is_dir()])

NUM_CLASSES = len(NEW_CLASS_NAMES)
CLASS_TO_ID = {name: i for i, name in enumerate(NEW_CLASS_NAMES)}

if NUM_CLASSES == 0:
    raise FileNotFoundError(f"No class folders found in: {DATA_DIR}. Please check the path and contents.")

print(f"✅ Automatically Detected {NUM_CLASSES} classes:")
print(NEW_CLASS_NAMES)
print(f"\nModel will be saved to: {MODEL_SAVE_PATH}")

# --- 4. Collect All Image Paths and Labels ---
print("\n--- Collecting and mapping image files ---")
all_paths = []
all_labels = []

for class_name, class_id in CLASS_TO_ID.items():
    class_folder = DATA_DIR / class_name
    class_paths = glob.glob(str(class_folder / '*.*'))
    class_paths = [p for p in class_paths if p.lower().endswith(('.jpg', '.jpeg', '.png'))]

    if len(class_paths) == 0:
        print(f"⚠️ Warning: No images found for class: {class_name}. Skipping.")
        continue

    all_paths.extend(class_paths)
    all_labels.extend([class_id] * len(class_paths))

print(f"Total images found across all classes: {len(all_paths)}")
if len(all_paths) == 0:
    raise FileNotFoundError("No images found in total. Please check subfolder contents.")

# --- 5. Stratified Split ---
train_paths, val_paths, train_labels, val_labels = train_test_split(
    all_paths,
    all_labels,
    test_size=VALIDATION_SPLIT,
    random_state=RANDOM_SEED,
    shuffle=True,
    stratify=all_labels
)

print(f"Train samples: {len(train_paths)}, Validation samples: {len(val_paths)}")

# --- 6. Create YOLO Directory Structure (First, before writing files) ---
# Clean up previous runs
if os.path.exists(YOLO_DATA_ROOT):
    shutil.rmtree(YOLO_DATA_ROOT)

os.makedirs(YOLO_DATA_ROOT, exist_ok=True)
os.makedirs(os.path.join(YOLO_DATA_ROOT, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(YOLO_DATA_ROOT, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(YOLO_DATA_ROOT, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(YOLO_DATA_ROOT, 'labels', 'val'), exist_ok=True)
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True) # Create 'mod' folder

print("YOLO directory structure and model save path created.")

# --- 7. Generate YOLO Files (Image Copy & Placeholder Labels) ---
PLACEHOLDER_BOX = "0.5 0.5 1.0 1.0"

def generate_yolo_files(paths, labels, split):
    for path, label in zip(paths, labels):
        # 1. Image Copy
        base_name = os.path.basename(path)
        dest_img_path = os.path.join(YOLO_DATA_ROOT, 'images', split, base_name)
        shutil.copy(path, dest_img_path)

        # 2. Label File Creation
        label_filename = os.path.splitext(base_name)[0] + '.txt'
        dest_label_path = os.path.join(YOLO_DATA_ROOT, 'labels', split, label_filename)

        yolo_label_line = f"{label} {PLACEHOLDER_BOX}\n"

        with open(dest_label_path, 'w') as f:
            f.write(yolo_label_line)

generate_yolo_files(train_paths, train_labels, 'train')
generate_yolo_files(val_paths, val_labels, 'val')

print("✅ YOLO-formatted image and label files successfully created.")

# --- 8. Create YOLO Config File (data.yaml) ---
data_yaml = {
    'train': os.path.join(YOLO_DATA_ROOT, 'images', 'train'),
    'val': os.path.join(YOLO_DATA_ROOT, 'images', 'val'),
    'nc': NUM_CLASSES,
    'names': NEW_CLASS_NAMES
}

yaml_file_path = os.path.join(YOLO_DATA_ROOT, 'data.yaml')
with open(yaml_file_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print(f"✅ YOLO config file saved to {yaml_file_path}")

# --- 9. Create Class Names Text File (classes.txt) ---
classes_file_path = os.path.join(YOLO_DATA_ROOT, 'classes.txt')
with open(classes_file_path, 'w') as f:
    for class_name in NEW_CLASS_NAMES:
        f.write(f"{class_name}\n")

print(f"✅ Class labels file saved to {classes_file_path}")

Mounted at /content/drive
✅ Automatically Detected 9 classes:
['Great_Hornbill', 'Indian_Elephant', 'King_Cobra', 'Leopard', 'Monitor_Lizard', 'Nilgiri_Langur', 'Sloth_Bear', 'Spotted_Deer_Chital', 'Wild_Boar']

Model will be saved to: /content/drive/MyDrive/mod/animal_classifier_9class.keras

--- Collecting and mapping image files ---
Total images found across all classes: 1909
Train samples: 1527, Validation samples: 382
YOLO directory structure and model save path created.
✅ YOLO-formatted image and label files successfully created.
✅ YOLO config file saved to /content/yolo_dataset/data.yaml
✅ Class labels file saved to /content/yolo_dataset/classes.txt


In [ ]:
# --- Path Overwrite and Verification Code (RUN THIS NOW) ---

import glob
import os
import numpy as np

YOLO_DATA_ROOT = '/content/yolo_dataset' # Ensure this variable is defined

# 1. Load the new, local paths from the disk
local_train_paths = glob.glob(os.path.join(YOLO_DATA_ROOT, 'images', 'train', '*.*'))
local_val_paths = glob.glob(os.path.join(YOLO_DATA_ROOT, 'images', 'val', '*.*'))

# 2. Extract the class ID from the file names to get the correct labels back
# We can't rely on the old train_labels/val_labels lists because their indexing might be wrong now.
# We'll use the YOLO label files we generated in Cell 1 to get the true label for each image.

def get_labels_from_yolo_files(paths, split):
    labels = []
    for path in paths:
        # e.g., /content/yolo_dataset/images/train/img_123.jpg -> img_123.txt
        base_name = os.path.basename(path)
        label_filename = os.path.splitext(base_name)[0] + '.txt'
        label_path = os.path.join(YOLO_DATA_ROOT, 'labels', split, label_filename)

        try:
            with open(label_path, 'r') as f:
                # The first number in the YOLO file is the class ID
                label = int(f.readline().split()[0])
                labels.append(label)
        except FileNotFoundError:
            # This should not happen if Cell 1 completed, but good for stability
            print(f"Error: Label file not found for {path}")
            labels.append(-1) # Use a flag for error

    return np.array(labels)

# Overwrite global variables with local paths and their corresponding labels
train_paths = local_train_paths
val_paths = local_val_paths
train_labels = get_labels_from_yolo_files(train_paths, 'train')
val_labels = get_labels_from_yolo_files(val_paths, 'val')

print("✅ Paths and Labels successfully reset to stable local YOLO directory using label files.")
print(f"Verified {len(train_paths)} local training image paths.")
print(f"Verified {len(val_paths)} local validation image paths.")

✅ Paths and Labels successfully reset to stable local YOLO directory using label files.
Verified 638 local training image paths.
Verified 285 local validation image paths.


guarenteed pipe line


In [ ]:
# --- STEP 1: Final Path Reset and Label Extraction (Guaranteed Local Paths) ---

import glob
import os
import numpy as np
import tensorflow as tf

YOLO_DATA_ROOT = '/content/yolo_dataset'

def get_labels_from_yolo_files(paths, split):
    """Reads the class ID from the corresponding YOLO label file."""
    labels = []
    for path in paths:
        base_name = os.path.basename(path)
        label_filename = os.path.splitext(base_name)[0] + '.txt'
        label_path = os.path.join(YOLO_DATA_ROOT, 'labels', split, label_filename)

        try:
            with open(label_path, 'r') as f:
                label = int(f.readline().split()[0])
                labels.append(label)
        except Exception as e:
            # If we fail to read the label, we skip the path entirely
            print(f"Skipping image due to error: {e}")
            continue

    return np.array(labels)

# 1. Load the new, local paths
train_paths_stable = glob.glob(os.path.join(YOLO_DATA_ROOT, 'images', 'train', '*.*'))
val_paths_stable = glob.glob(os.path.join(YOLO_DATA_ROOT, 'images', 'val', '*.*'))

# 2. Extract labels using the local label files
train_labels_stable = get_labels_from_yolo_files(train_paths_stable, 'train')
val_labels_stable = get_labels_from_yolo_files(val_paths_stable, 'val')

# Filter out any paths that failed to load a label
train_paths_stable = [p for p, l in zip(train_paths_stable, train_labels_stable) if l != -1]
val_paths_stable = [p for p, l in zip(val_paths_stable, val_labels_stable) if l != -1]
train_labels_stable = train_labels_stable[train_labels_stable != -1]
val_labels_stable = val_labels_stable[val_labels_stable != -1]

print(f"Final STABLE Train Samples: {len(train_paths_stable)}, Val Samples: {len(val_paths_stable)}")

# --- STEP 2: Dataset Creation (Rebuild with Stable Paths) ---

IMG_SIZE = 224
BATCH_SIZE = 32
RANDOM_SEED = 42

def preprocess_image(file_path, label):
    """The function where the ReadFile error occurs. Must use correct path here."""
    img = tf.io.read_file(file_path) # <--- This is where the error originates
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.cast(img, tf.float32)
    return img, label

def create_dataset(paths, labels, shuffle=False):
    # Pass stable paths (converted to standard string list) and labels
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels.astype(np.int32)))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000, seed=RANDOM_SEED)
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

# REBUILD the core datasets!
train_ds = create_dataset(train_paths_stable, train_labels_stable, shuffle=True)
val_ds = create_dataset(val_paths_stable, val_labels_stable, shuffle=False)
print("✅ Core TensorFlow Datasets REBUILT with stable local paths.")

# --- STEP 3: Augmentation and Weights (Rebuild with Stable Data) ---

from tensorflow.keras import layers
from sklearn.utils import class_weight

# Calculate new class weights based on the STABLE labels
unique_classes = np.unique(train_labels_stable)
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=unique_classes,
    y=train_labels_stable
)
class_weights = dict(zip(unique_classes, weights))
print("\n--- Recalculated Class Weights (STABLE) ---")
print(class_weights)

# Re-define Augmentation (from Cell 3)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal", seed=RANDOM_SEED),
    layers.RandomRotation(0.1, seed=RANDOM_SEED),
    layers.RandomZoom(0.1, seed=RANDOM_SEED),
    layers.RandomContrast(0.1, seed=RANDOM_SEED),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1, seed=RANDOM_SEED),
], name="data_augmentation")

def augment_and_preprocess(image, label):
    image = data_augmentation(image, training=True)
    return image, label

# REBUILD the augmented dataset
augmented_train_ds = train_ds.map(augment_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
print("✅ Augmented training dataset REBUILT.")

# --- STEP 4: Stage 1 Training (Final Attempt) ---

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model # Assume model from Cell 2 is in memory

# 1. Define Callbacks
checkpoint_filepath = os.path.join(os.path.dirname('/content/drive/MyDrive/mod/animal_classifier_9class.keras'), 'stage1_best_head.keras')

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min',
    restore_best_weights=True,
    verbose=1
)
callbacks_list = [model_checkpoint_callback, early_stopping_callback]

# 2. Ensure model is compiled (use the 'model' object from Cell 2)
model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\nStarting Stage 1: FINAL TRAINING ATTEMPT...")

# 3. Run Training
history_stage1 = model.fit(
    augmented_train_ds,
    epochs=25,
    validation_data=val_ds,
    class_weight=class_weights,
    callbacks=callbacks_list
)

# 4. Save Final Stage 1 Weights
model.save('/content/drive/MyDrive/mod/animal_classifier_9class.keras')
print("\n✅ Training Complete. Proceed to Stage 2: Fine-Tuning.")

Final STABLE Train Samples: 638, Val Samples: 285
✅ Core TensorFlow Datasets REBUILT with stable local paths.

--- Recalculated Class Weights (STABLE) ---
{np.int64(0): np.float64(2.2152777777777777), np.int64(1): np.float64(0.22648207312744054), np.int64(2): np.float64(3.5444444444444443), np.int64(3): np.float64(1.0126984126984127), np.int64(4): np.float64(1.162112932604736), np.int64(5): np.float64(2.084967320261438), np.int64(6): np.float64(1.288888888888889), np.int64(7): np.float64(1.7722222222222221), np.int64(8): np.float64(5.452991452991453)}
✅ Augmented training dataset REBUILT.

Starting Stage 1: FINAL TRAINING ATTEMPT...
Epoch 1/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5794 - loss: 8.8160   
Epoch 1: val_loss improved from inf to 6.30406, saving model to /content/drive/MyDrive/mod/stage1_best_head.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - accuracy: 0.5886 - loss: 8.7767 - val_accuracy: 0.9509 - val_loss: 6.3041
Epoch 2/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 716

stage 2 training

In [ ]:
# Quick Check to Identify the Base Model Layer Name and Index
print(model.summary(expand_nested=True, show_trainable=True))

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)      ┃ Output Shape    ┃   Param # ┃ Connected to   ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_2     │ (None, 224,     │         0 │ -              │   -   │
│ (InputLayer)      │ 224, 3)         │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ rescaling_2       │ (None, 224,     │         0 │ input_layer_2… │   -   │
│ (Rescaling)       │ 224, 3)         │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ normalization_1   │ (None, 224,     │         7 │ rescaling_2[0… │   N   │
│ (Normalization)   │ 224, 3)         │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ rescaling_3       │ (None, 224,     │         0 │ normalization… │   -   │
│ (Rescaling)       │ 224, 3)         │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ stem_conv_pad     │ (None, 225,     │         0 │ rescaling_3[0… │   -   │
│ (ZeroPadding2D)   │ 225, 3)         │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ stem_conv         │ (None, 112,     │       864 │ stem_conv_pad… │   N   │
│ (Conv2D)          │ 112, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ stem_bn           │ (None, 112,     │       128 │ stem_conv[0][… │   N   │
│ (BatchNormalizat… │ 112, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ stem_activation   │ (None, 112,     │         0 │ stem_bn[0][0]  │   -   │
│ (Activation)      │ 112, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ block1a_dwconv    │ (None, 112,     │       288 │ stem_activati… │   N   │
│ (DepthwiseConv2D) │ 112, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ block1a_bn        │ (None, 112,     │       128 │ block1a_dwcon… │   N   │
│ (BatchNormalizat… │ 112, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ block1a_activati… │ (None, 112,     │         0 │ block1a_bn[0]… │   -   │
│ (Activation)      │ 112, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ block1a_se_squee… │ (None, 32)      │         0 │ block1a_activ… │   -   │
│ (GlobalAveragePo… │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ block1a_se_resha… │ (None, 1, 1,    │         0 │ block1a_se_sq… │   -   │
│ (Reshape)         │ 32)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ block1a_se_reduce │ (None, 1, 1, 8) │       264 │ block1a_se_re… │   N   │
│ (Conv2D)          │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ block1a_se_expand │ (None, 1, 1,    │       288 │ block1a_se_re… │   N   │
│ (Conv2D)          │ 32)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ block1a_se_excite │ (None, 112,     │         0 │ block1a_activ… │   -   │
│ (Multiply)        │ 112, 32)        │           │ block1a_se_ex… │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ block1a_project_… │ (None, 112,     │       512 │ block1a_se_ex… │   N 

 Total params: 6,031,040 (23.01 MB)

 Trainable params: 660,489 (2.52 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

 Optimizer params: 1,320,980 (5.04 MB)

None


In [ ]:
print(final_model.summary(expand_nested=True, show_trainable=True))

NameError: name 'final_model' is not defined

re building the model and fine tuning the model

In [ ]:
# Cell 5 (Final): Rebuild Model, Load Weights, and Fine-Tune (FINAL ROBUST FIX)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import tensorflow as tf

# --- Configuration (Based on your successful Cell 1/Merged Cell data) ---
MODEL_SAVE_PATH = '/content/drive/MyDrive/mod/animal_classifier_9class.keras'
FINAL_CHECKPOINT_FILEPATH = os.path.join(os.path.dirname(MODEL_SAVE_PATH), 'final_best_fine_tuned.keras')

# --- 1. Model is Rebuilt and Weights Loaded (Assuming prior steps succeeded) ---
print("Model loaded successfully. Preparing for fine-tuning...")


# --- 2. Configure Model for Stage 2 Fine-Tuning ---

# We need to unfreeze the LAST N layers of the *entire* model.
# Total layers in the model (including the new head layers)
total_layers = len(model.layers)
# We want to unfreeze the last 40 layers, which includes the classification head.
UNFREEZE_LAYERS_COUNT = 40
UNFREEZE_START_INDEX = total_layers - UNFREEZE_LAYERS_COUNT

print(f"Total layers in model: {total_layers}")

# 2a. Freeze ALL layers first
for layer in model.layers:
    layer.trainable = False

# 2b. Unfreeze the last N layers
for layer in model.layers[UNFREEZE_START_INDEX:]:
    layer.trainable = True

trainable_count = sum([layer.trainable for layer in model.layers])
print(f"Total trainable layers (Top {UNFREEZE_LAYERS_COUNT} + Head): {trainable_count}")


# --- 3. Re-compile with a Low Learning Rate ---

model.compile(
    optimizer=Adam(learning_rate=1e-5), # Low learning rate (0.00001) for fine-tuning
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\nStarting Stage 2: Fine-Tuning the Top Layers by Absolute Index...")


# --- 4. Define and Run Training ---

model_checkpoint_callback_stage2 = ModelCheckpoint(
    filepath=FINAL_CHECKPOINT_FILEPATH,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

early_stopping_callback_stage2 = EarlyStopping(
    monitor='val_loss',
    patience=8,
    mode='min',
    restore_best_weights=True,
    verbose=1
)
callbacks_list_stage2 = [model_checkpoint_callback_stage2, early_stopping_callback_stage2]

# Run Training (using augmented_train_ds, val_ds, and class_weights from prior cells)
history_stage2 = model.fit(
    augmented_train_ds,
    epochs=20,
    validation_data=val_ds,
    class_weight=class_weights,
    callbacks=callbacks_list_stage2
)

# Save Final Model
model.save(MODEL_SAVE_PATH)
print(f"\n✅ Stage 2 Training Complete. Final best model saved to: {FINAL_CHECKPOINT_FILEPATH}")

Model loaded successfully. Preparing for fine-tuning...
Total layers in model: 243
Total trainable layers (Top 40 + Head): 40

Starting Stage 2: Fine-Tuning the Top Layers by Absolute Index...
Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6231 - loss: 1.1071   
Epoch 1: val_loss improved from inf to 0.72965, saving model to /content/drive/MyDrive/mod/final_best_fine_tuned.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.6226 - loss: 1.1045 - val_accuracy: 0.9509 - val_loss: 0.7297
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 685ms/step - accuracy: 0.6575 - loss: 0.9909
Epoch 2: val_loss did not improve from 0.72965
20/20 ━━━━━━━━━━━━━━━━━━━━ 18s 856ms/step - accuracy: 0.6576 - loss: 0.9914 - val_accuracy: 0.9474 - val_loss: 0.7625
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 706ms/step - accuracy: 0.7102 - loss: 0.8842
Epoch 3: val_loss did not improve from 0.72965
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 893ms/step - accuracy: 0.7109 - loss: 0.8884 - val_accuracy: 0.9368 - va

evaluation model

In [ ]:
# Cell 6: Final Evaluation, Metrics Saving, and Credential Consolidation

import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report, confusion_matrix
import json

# --- Configuration (using global variables) ---
MODEL_SAVE_PATH = '/content/drive/MyDrive/mod/animal_classifier_9class.keras'
FINAL_CHECKPOINT_FILEPATH = os.path.join(os.path.dirname(MODEL_SAVE_PATH), 'final_best_fine_tuned.keras')
MOD_FOLDER = os.path.dirname(MODEL_SAVE_PATH) # '/content/drive/MyDrive/mod'

# --- 1. Load the Best Performing Model ---
# We load the best model saved during Stage 2, which has the optimal weights.
final_model = tf.keras.models.load_model(FINAL_CHECKPOINT_FILEPATH)
print(f"✅ Loaded final best model from: {FINAL_CHECKPOINT_FILEPATH}")

# --- 2. Evaluate Model on Validation Data ---
print("\n--- Final Model Evaluation ---")
loss, accuracy = final_model.evaluate(val_ds, verbose=1)

print(f"\nFinal Validation Loss: {loss:.4f}")
print(f"Final Validation Accuracy: {accuracy:.4f}")

# --- 3. Detailed Performance Metrics (Classification Report) ---

# Get true labels and predictions
# Note: This step can be memory intensive
y_true = np.concatenate([y.numpy() for x, y in val_ds], axis=0)
y_pred_probs = final_model.predict(val_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\n--- Classification Report ---")
# NEW_CLASS_NAMES was defined in Cell 1 (e.g., ['Great_Hornbill', ...])
report = classification_report(y_true, y_pred, target_names=NEW_CLASS_NAMES, output_dict=True)
print(classification_report(y_true, y_pred, target_names=NEW_CLASS_NAMES))

# --- 4. Save All Credentials/Artifacts to Drive ---

# 4a. Save Metrics to JSON
metrics_path = os.path.join(MOD_FOLDER, 'final_metrics.json')
with open(metrics_path, 'w') as f:
    json.dump({
        'validation_accuracy': accuracy,
        'validation_loss': loss,
        'classification_report': report
    }, f, indent=4)
print(f"\n✅ Final metrics saved to: {metrics_path}")


# 4b. Save Confusion Matrix (for detailed analysis)
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, index=NEW_CLASS_NAMES, columns=NEW_CLASS_NAMES)
cm_path = os.path.join(MOD_FOLDER, 'confusion_matrix.csv')
cm_df.to_csv(cm_path)
print(f"✅ Confusion Matrix saved to: {cm_path}")


# 4c. Save Class Names (The definitive label-to-ID mapping)
# This is crucial for deployment/inference
class_names_path = os.path.join(MOD_FOLDER, 'classes.txt')
with open(class_names_path, 'w') as f:
    for name in NEW_CLASS_NAMES:
        f.write(f"{name}\n")
print(f"✅ Class names saved to: {class_names_path}")


# --- 5. Final Model Confirmation ---
# The absolute best model is already saved at FINAL_CHECKPOINT_FILEPATH.
# MODEL_SAVE_PATH is a convenience symlink/copy.

print("\nAll training and evaluation artifacts are secured on Google Drive.")

ValueError: File not found: filepath=/content/drive/MyDrive/mod/final_best_fine_tuned.keras. Please ensure the file is an accessible `.keras` zip file.

testing the model with images

In [ ]:
# Cell 7: Full Prediction and Class Mapping Test

import tensorflow as tf
import numpy as np
import os
import random

# --- Configuration ---
MODEL_SAVE_PATH = '/content/drive/MyDrive/mod/animal_classifier_9class.keras'
FINAL_CHECKPOINT_FILEPATH = os.path.join(os.path.dirname(MODEL_SAVE_PATH), 'final_best_fine_tuned.keras')
# Assuming NEW_CLASS_NAMES list is still defined from Cell 1/Merged Cell

# --- 1. Load the Best Model ---
final_model = tf.keras.models.load_model(FINAL_CHECKPOINT_FILEPATH)
print(f"✅ Loaded final best model from: {FINAL_CHECKPOINT_FILEPATH}")

# --- 2. Generate Predictions ---
print("\n--- Generating predictions on validation data... ---")

# Predict on the entire validation dataset
y_pred_probs = final_model.predict(val_ds)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Retrieve true labels (y_true was already calculated in the previous cell, but re-calculate for safety)
y_true_labels = np.concatenate([y.numpy() for x, y in val_ds], axis=0)

# --- 3. Display Sample Results ---

print("\n--- Sample Prediction Results (True vs. Predicted) ---")
print("{:<18} {:<18} {:<10}".format("True Class", "Predicted Class", "Confidence"))
print("-" * 50)

# Create a list of sample indices to display (e.g., 10 random samples)
sample_indices = random.sample(range(len(y_true_labels)), 10)

for i in sample_indices:
    true_id = y_true_labels[i]
    pred_id = y_pred_classes[i]

    # Get the confidence level of the predicted class
    confidence = y_pred_probs[i][pred_id]

    true_name = NEW_CLASS_NAMES[true_id]
    pred_name = NEW_CLASS_NAMES[pred_id]

    # Print the results
    print("{:<18} {:<18} {:.4f}".format(true_name, pred_name, confidence))

# --- 4. Final Confirmation ---

print(f"\n✅ Prediction test complete. Model successfully mapped {len(y_pred_classes)} images across {len(NEW_CLASS_NAMES)} classes.")
print("The model output is confirmed to be stable.")

✅ Loaded final best model from: /content/drive/MyDrive/mod/final_best_fine_tuned.keras

--- Generating predictions on validation data... ---
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step

--- Sample Prediction Results (True vs. Predicted) ---
True Class         Predicted Class    Confidence
--------------------------------------------------
Spotted_Deer_Chital Spotted_Deer_Chital 0.8095
Monitor_Lizard     Monitor_Lizard     0.9629
Leopard            Leopard            0.9497
Indian_Elephant    Indian_Elephant    0.9958
Sloth_Bear         Sloth_Bear         0.6891
Sloth_Bear         Sloth_Bear         0.9624
Leopard            Leopard            0.8760
Great_Hornbill     Great_Hornbill     0.9446
Sloth_Bear         Sloth_Bear         0.8737
Indian_Elephant    Indian_Elephant    0.9617

✅ Prediction test complete. Model successfully mapped 285 images across 9 classes.
The model output is confirmed to be stable.


testing with video

In [ ]:
# Cell 8: Video Inference and Real-time Prediction Overlay

import cv2
import numpy as np
import tensorflow as tf
import os
import time
from google.colab.patches import cv2_imshow # Special function for Colab display

# --- 1. Configuration ---

# Path to your final best model
MODEL_PATH = '/content/drive/MyDrive/mod/final_best_fine_tuned.keras'
# Path to the input video file (CHANGE THIS to your video location)
INPUT_VIDEO_PATH = '/content/drive/MyDrive/test/test_video3.mp4'
# Path for the output video file
OUTPUT_VIDEO_PATH = '/content/drive/MyDrive/mod/output_video_predictions2.mp4'

IMG_SIZE = 224 # Model input size
CONFIDENCE_THRESHOLD = 0.7 # Only display prediction if confidence is above this threshold

# Load the class names list
# Assuming NEW_CLASS_NAMES is globally available, if not:
# with open('/content/drive/MyDrive/mod/classes.txt', 'r') as f:
#     NEW_CLASS_NAMES = [line.strip() for line in f.readlines()]

# --- 2. Load Model and Video ---

print(f"Loading model from: {MODEL_PATH}")
final_model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model loaded successfully.")

# Open the video file
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

if not cap.isOpened():
    raise IOError(f"Cannot open video file: {INPUT_VIDEO_PATH}")

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the video writer to save the output video
# We use XVID codec for broad compatibility
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))
print(f"🎥 Video stream opened. Frame rate: {fps:.2f} FPS.")


# --- 3. Frame Processing Loop ---

frame_count = 0
start_time = time.time()

while True:
    ret, frame = cap.read()

    if not ret:
        break # End of video stream

    frame_count += 1

    # 3a. Prepare Frame for Model Prediction
    # Resize frame to model input size (224x224)
    img_tensor = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    # Convert BGR (OpenCV format) to RGB (Model expectation, usually) and normalize to [0, 255]
    # EfficientNet preprocessing layer handles normalization internally if input is [0, 255].
    img_tensor = cv2.cvtColor(img_tensor, cv2.COLOR_BGR2RGB)
    # Add batch dimension: (224, 224, 3) -> (1, 224, 224, 3)
    img_tensor = np.expand_dims(img_tensor, axis=0)

    # 3b. Predict
    predictions = final_model.predict(img_tensor, verbose=0)
    pred_class_id = np.argmax(predictions[0])
    confidence = predictions[0][pred_class_id]

    # 3c. Overlay Prediction on the Original Frame

    label = "Predicting..."
    color = (0, 0, 255) # Red for low confidence/unclassified

    if confidence >= CONFIDENCE_THRESHOLD:
        pred_name = NEW_CLASS_NAMES[pred_class_id]
        label = f"{pred_name}: {confidence:.2f}"
        color = (0, 255, 0) # Green for high confidence

    # Add text label to the frame (bottom left corner)
    cv2.putText(frame, label, (10, frame_height - 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

    # 3d. Write the frame to the output video
    out.write(frame)

# --- 4. Cleanup and Summary ---

end_time = time.time()
processing_fps = frame_count / (end_time - start_time)

cap.release()
out.release()

print("\n--- Inference Summary ---")
print(f"Total frames processed: {frame_count}")
print(f"Processing Speed: {processing_fps:.2f} FPS")
print(f"✅ Output video with predictions saved to: {OUTPUT_VIDEO_PATH}")

Loading model from: /content/drive/MyDrive/mod/final_best_fine_tuned.keras
✅ Model loaded successfully.
🎥 Video stream opened. Frame rate: 30.00 FPS.

--- Inference Summary ---
Total frames processed: 1349
Processing Speed: 2.48 FPS
✅ Output video with predictions saved to: /content/drive/MyDrive/mod/output_video_predictions2.mp4


praying if it would work

In [ ]:
# Cell 8 (Final): Batch Video Inference on Test Folder

import cv2
import numpy as np
import tensorflow as tf
import os
import time

# --- 1. Configuration (UPDATE THESE PATHS) ---

# Path to your final best model
MODEL_PATH = '/content/drive/MyDrive/mod/final_best_fine_tuned.keras'
# Directory containing all your test videos
INPUT_VIDEO_DIR = '/content/drive/MyDrive/test/'
# Directory where output videos will be saved
OUTPUT_VIDEO_DIR = '/content/drive/MyDrive/mod/video_predictions/'

IMG_SIZE = 224 # Model input size
CONFIDENCE_THRESHOLD = 0.7 # Minimum confidence to display a class
# Assuming NEW_CLASS_NAMES list is still defined globally

# --- 2. Setup ---

# Ensure the output directory exists
os.makedirs(OUTPUT_VIDEO_DIR, exist_ok=True)
print(f"Loading model from: {MODEL_PATH}")
final_model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model loaded successfully.")

# Get list of all video files
video_files = [f for f in os.listdir(INPUT_VIDEO_DIR) if f.endswith('.mp4') or f.endswith('.avi')]

if not video_files:
    raise FileNotFoundError(f"No video files (.mp4 or .avi) found in: {INPUT_VIDEO_DIR}")

print(f"\n🎥 Found {len(video_files)} videos to process.")


# --- 3. Batch Processing Loop ---

def process_video(input_path, output_dir, model, class_names):
    """Processes a single video file for inference."""
    cap = cv2.VideoCapture(input_path)

    if not cap.isOpened():
        print(f"⚠️ Warning: Could not open video file: {input_path}")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define output path
    video_filename = os.path.basename(input_path)
    output_filename = f"{os.path.splitext(video_filename)[0]}_predictions.mp4"
    output_path = os.path.join(output_dir, output_filename)

    # Define the video writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    start_time = time.time()

    print(f"   -> Processing '{video_filename}' (FPS: {fps:.2f})...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Prepare Frame for Prediction
        img_tensor = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        img_tensor = cv2.cvtColor(img_tensor, cv2.COLOR_BGR2RGB)
        img_tensor = np.expand_dims(img_tensor, axis=0)

        # Predict
        predictions = model.predict(img_tensor, verbose=0)
        pred_class_id = np.argmax(predictions[0])
        confidence = predictions[0][pred_class_id]

        # Overlay Prediction
        label = "Unclassified"
        color = (0, 0, 255) # Red

        if confidence >= CONFIDENCE_THRESHOLD:
            pred_name = class_names[pred_class_id]
            label = f"{pred_name}: {confidence:.2f}"
            color = (0, 255, 0) # Green

        # Add text label
        cv2.putText(frame, label, (10, frame_height - 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

        # Write the frame
        out.write(frame)

    # Cleanup and Summary
    cap.release()
    out.release()

    end_time = time.time()
    processing_fps = frame_count / (end_time - start_time)
    print(f"   -> Completed. Processing Speed: {processing_fps:.2f} FPS. Output saved to {output_path}")

# Iterate over all found video files
for video_file in video_files:
    input_video_path = os.path.join(INPUT_VIDEO_DIR, video_file)
    process_video(input_video_path, OUTPUT_VIDEO_DIR, final_model, NEW_CLASS_NAMES)


print("\n✅ Batch video inference complete. Check the 'video_predictions' subfolder in your 'mod' directory.")

Loading model from: /content/drive/MyDrive/mod/final_best_fine_tuned.keras
✅ Model loaded successfully.

🎥 Found 9 videos to process.
   -> Processing 'test_video.mp4' (FPS: 30.00)...
   -> Completed. Processing Speed: 8.38 FPS. Output saved to /content/drive/MyDrive/mod/video_predictions/test_video_predictions.mp4
   -> Processing 'Great hornbill sound 🧡 [QA54hEMgOps].mp4' (FPS: 30.00)...
   -> Completed. Processing Speed: 12.30 FPS. Output saved to /content/drive/MyDrive/mod/video_predictions/Great hornbill sound 🧡 [QA54hEMgOps]_predictions.mp4
   -> Processing 'Sloth Bear Attack [npC1XpHQFB0].mp4' (FPS: 29.87)...
   -> Completed. Processing Speed: 12.98 FPS. Output saved to /content/drive/MyDrive/mod/video_predictions/Sloth Bear Attack [npC1XpHQFB0]_predictions.mp4
   -> Processing 'Lizard Scrambles Across Tile Floor _ ViralHog [7PlZc_xqRmk].mp4' (FPS: 30.00)...
   -> Completed. Processing Speed: 0.00 FPS. Output saved to /content/drive/MyDrive/mod/video_predictions/Lizard Scrambles

wish me luck

In [ ]:
# Cell 8 (Final): Batch Video Inference on Test Folder (REVISED FOR MISSING CLASS NAMES)

import cv2
import numpy as np
import tensorflow as tf
import os
import time

# --- 1. Configuration (UPDATE THESE PATHS) ---

# Path to your final best model
MODEL_PATH = '/content/drive/MyDrive/mod/final_best_fine_tuned.keras'
# Directory containing all your test videos
INPUT_VIDEO_DIR = '/content/drive/MyDrive/test/'
# Directory where output videos will be saved
OUTPUT_VIDEO_DIR = '/content/drive/MyDrive/mod/video_predictions/'
# Path to the saved class names file
CLASSES_PATH = '/content/drive/MyDrive/mod/classes.txt'

IMG_SIZE = 224
CONFIDENCE_THRESHOLD = 0.7

# --- 2. Setup: Load Model and Class Names ---

os.makedirs(OUTPUT_VIDEO_DIR, exist_ok=True)

# 2a. Load Model
print(f"Loading model from: {MODEL_PATH}")
final_model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model loaded successfully.")

# 2b. Load Class Names (THE FIX)
print(f"Loading class names from: {CLASSES_PATH}")
try:
    with open(CLASSES_PATH, 'r') as f:
        NEW_CLASS_NAMES = [line.strip() for line in f.readlines()]
    print(f"✅ Loaded {len(NEW_CLASS_NAMES)} class names: {NEW_CLASS_NAMES}")
except FileNotFoundError:
    raise FileNotFoundError(f"Error: Class names file not found at {CLASSES_PATH}. Cannot proceed.")


# 2c. Get Video Files
video_files = [f for f in os.listdir(INPUT_VIDEO_DIR) if f.endswith('.mp4') or f.endswith('.avi')]

if not video_files:
    raise FileNotFoundError(f"No video files (.mp4 or .avi) found in: {INPUT_VIDEO_DIR}")

print(f"\n🎥 Found {len(video_files)} videos to process.")


# --- 3. Batch Processing Loop ---

def process_video(input_path, output_dir, model, class_names):
    """Processes a single video file for inference with robust codec."""
    print(f"\nAttempting to open input video: {input_path}")
    cap = cv2.VideoCapture(input_path)

    if not cap.isOpened():
        print(f"❌ ERROR: Could not open video file: {input_path}")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define output path
    video_filename = os.path.basename(input_path)
    output_filename = f"{os.path.splitext(video_filename)[0]}_predictions.mp4"
    output_path = os.path.join(output_dir, output_filename)

    # Define the video writer using MP4V codec
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Check if VideoWriter initialized successfully
    if not out.isOpened():
        print(f"❌ ERROR: VideoWriter failed to initialize for {output_path}. Try installing codecs with 'apt install -y ffmpeg libsm6 libxext6' and retry.")
        cap.release()
        return

    frame_count = 0
    start_time = time.time()

    print(f"   -> Processing '{video_filename}' (FPS: {fps:.2f})...")

    while True:
        ret, frame = cap.read()

        if not ret:
            print(f"   -> End of stream reached or frame read failed after {frame_count} frames.")
            break

        # --- Inference Logic ---
        frame_count += 1
        img_tensor = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        img_tensor = cv2.cvtColor(img_tensor, cv2.COLOR_BGR2RGB)
        img_tensor = np.expand_dims(img_tensor, axis=0)

        predictions = model.predict(img_tensor, verbose=0)
        pred_class_id = np.argmax(predictions[0])
        confidence = predictions[0][pred_class_id]

        label = "Unclassified"
        color = (0, 0, 255)

        if confidence >= CONFIDENCE_THRESHOLD:
            pred_name = class_names[pred_class_id]
            label = f"{pred_name}: {confidence:.2f}"
            color = (0, 255, 0)

        cv2.putText(frame, label, (10, frame_height - 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

        out.write(frame)

    # Cleanup and Summary
    cap.release()
    out.release()

    end_time = time.time()
    processing_fps = frame_count / (end_time - start_time)
    print(f"   -> Completed. Frames: {frame_count}. Processing Speed: {processing_fps:.2f} FPS. Output saved to {output_path}")

# --- Main Execution ---
for video_file in video_files:
    input_video_path = os.path.join(INPUT_VIDEO_DIR, video_file)
    process_video(input_video_path, OUTPUT_VIDEO_DIR, final_model, NEW_CLASS_NAMES)


print("\n✅ Batch video inference complete. Check the 'video_predictions' subfolder in your 'mod' directory.")

Loading model from: /content/drive/MyDrive/mod/final_best_fine_tuned.keras
✅ Model loaded successfully.
Loading class names from: /content/drive/MyDrive/mod/classes.txt
✅ Loaded 9 class names: ['Great_Hornbill', 'Indian_Elephant', 'King_Cobra', 'Leopard', 'Monitor_Lizard', 'Nilgiri_Langur', 'Sloth_Bear', 'Spotted_Deer_Chital', 'Wild_Boar']

🎥 Found 9 videos to process.

Attempting to open input video: /content/drive/MyDrive/test/Great hornbill sound 🧡 [QA54hEMgOps].mp4
   -> Processing 'Great hornbill sound 🧡 [QA54hEMgOps].mp4' (FPS: 30.00)...
   -> End of stream reached or frame read failed after 120 frames.
   -> Completed. Frames: 120. Processing Speed: 2.32 FPS. Output saved to /content/drive/MyDrive/mod/video_predictions/Great hornbill sound 🧡 [QA54hEMgOps]_predictions.mp4

Attempting to open input video: /content/drive/MyDrive/test/Indian spectacled cobra snake on tree 🐍 #cobra #cobrarescue #indiansnake #spectacledcobra #chitra [YNRotIAJlko].mp4
   -> Processing 'Indian spectacle

In [ ]:
!apt update && apt install -y ffmpeg libsm6 libxext6

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,820 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,520 kB]
Get:14 h